In [12]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
#app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
#app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
app_train_def_4 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v4.csv')
app_test = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_test_pruebav1.csv')
#print(app_train_def_4.info())
print(app_test.shape)
print(app_train_def_4.shape)


2.1.3
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
(48744, 1111)
(307511, 1110)


In [13]:
test_ids = app_test['SK_ID_CURR'].copy()

KeyError: 'SK_ID_CURR'

In [2]:
#Train y Test
X = app_train_def_4.drop(columns = ['TARGET'])
y = app_train_def_4['TARGET']


X.columns = X.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
app_test.columns = app_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [10]:
app_test = app_test.reindex(columns=X.columns, fill_value=0)
test_ids = app_test['SK_ID_CURR'].copy()
print(app_test.shape)
print(X.shape)

KeyError: 'SK_ID_CURR'

In [3]:
best_params = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    'num_leaves': 39,        # Fijo
    'learning_rate': 0.0162622,   # Fijo
    'max_depth': -1,         # Fijo
    'bagging_fraction': 0.798114 ,
    'bagging_freq': 14,
    'feature_fraction': 0.206931,
    'max_bin': 230,
    'min_child_samples': 94,
    'min_child_weight': 0.0166739,
    'min_gain_to_split': 0.084112,
    'reg_alpha': 0.106233,
    'reg_lambda': 37.0253,
    'scale_pos_weight': 1,   # Fijo
    'is_unbalance': False,   # Fijo
}

final_model_lgb = LGBMClassifier(
    n_estimators=2000,
    boosting_type=best_params['boosting_type'],
    objective=best_params['objective'],
    metric=best_params['metric'],
    num_leaves=best_params['num_leaves'],
    learning_rate=best_params['learning_rate'],
    max_bin=best_params['max_bin'],
    max_depth=best_params['max_depth'],
    bagging_fraction=best_params['bagging_fraction'],
    bagging_freq=best_params['bagging_freq'],   
    feature_fraction=best_params['feature_fraction'],
    min_child_samples=best_params['min_child_samples'],
    min_child_weight=best_params['min_child_weight'],
    min_gain_to_split=best_params['min_gain_to_split'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    scale_pos_weight=1,
    is_unbalance=False,
    random_state=42
)

final_model_lgb.fit(X, y)

[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] min_gain_to_split is set=0.084112, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.084112
[LightGBM] [Warning] feature_fraction is set=0.206931, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.206931
[LightGBM] [Warning] bagging_fraction is set=0.798114, subsample=1.0 will be ignored. Current value: bagging_fraction=0.798114
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] min_gain_to_split is set=0.084112, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.084112
[LightGBM] [Warning] feature_fraction is set=0.206931, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.206931
[LightGBM] [Warning] bagging_fraction is set=0.798114, subsample=1.0 will be ignored. Current value: bagging_fraction

LGBMClassifier(bagging_fraction=0.798114, bagging_freq=14,
               feature_fraction=0.206931, is_unbalance=False,
               learning_rate=0.0162622, max_bin=230, metric='auc',
               min_child_samples=94, min_child_weight=0.0166739,
               min_gain_to_split=0.084112, n_estimators=2000, num_leaves=39,
               objective='binary', random_state=42, reg_alpha=0.106233,
               reg_lambda=37.0253, scale_pos_weight=1)

In [9]:
y_pred_proba = final_model_lgb.predict_proba(app_test)[:, 1]
from sklearn.metrics import roc_auc_score
#auc_test = roc_auc_score(y_test, y_pred_proba)
#print(f'AUC en test: {auc_test}')

submision = pd.DataFrame({
    'SK_ID_CURR': test_ids,
    'TARGET': y_pred_proba_test
})
submission.to_csv('mysubmission.csv', index=False)

[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] min_gain_to_split is set=0.084112, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.084112
[LightGBM] [Warning] feature_fraction is set=0.206931, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.206931
[LightGBM] [Warning] bagging_fraction is set=0.798114, subsample=1.0 will be ignored. Current value: bagging_fraction=0.798114


NameError: name 'test_ids' is not defined